# Skeleton Code for the Final Project

## Environment Setup

In [2]:
# Install langchain
!pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-openai langchain-chroma bs4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 607.0/607.0 kB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.7/407.7 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.9/296.9 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 20.3 MB/s eta 0:

### Set API key

In [ ]:
# Set API key
OPENAI_API_KEY="your_api_key"

### Prepare model

In [4]:
# Prepare model
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0, api_key=OPENAI_API_KEY)

## Data Preparation

### Download and unzip backup vector DB

*   Source dataset: [FinQA](https://finqasite.github.io/)
*   EDA for the FinQA dataset: https://medium.com/@sohilsharma1996/finqa-project-numerical-reasoning-over-financial-data-77a3fb4133d2

We will provide the augmented dataset with evaluation code later(~31st Oct.)

Then you should build your own vector DB for retrieval.

In [5]:
!wget https://github.com/GSDSAML/prompt-engineering-2024fall/raw/refs/heads/main/final_project/db.zip

--2024-10-23 17:30:01--  https://github.com/GSDSAML/prompt-engineering-2024fall/raw/refs/heads/main/final_project/db.zip
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/GSDSAML/prompt-engineering-2024fall/refs/heads/main/final_project/db.zip [following]
--2024-10-23 17:30:02--  https://raw.githubusercontent.com/GSDSAML/prompt-engineering-2024fall/refs/heads/main/final_project/db.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 91090068 (87M) [application/zip]
Saving to: ‘db.zip’

db.zip              100%[===================>]  86.87M   122MB/s    in 0.7s    

2024-10-23 17:30:03 (122 MB/s) - 

In [6]:
!unzip db.zip

Archive:  db.zip
   creating: db/
  inflating: db/chroma.sqlite3       
   creating: db/c52496c4-34d2-4e9a-a78e-3c563b0247ff/
  inflating: db/c52496c4-34d2-4e9a-a78e-3c563b0247ff/data_level0.bin  
  inflating: db/c52496c4-34d2-4e9a-a78e-3c563b0247ff/header.bin  
  inflating: db/c52496c4-34d2-4e9a-a78e-3c563b0247ff/index_metadata.pickle  
  inflating: db/c52496c4-34d2-4e9a-a78e-3c563b0247ff/length.bin  
  inflating: db/c52496c4-34d2-4e9a-a78e-3c563b0247ff/link_lists.bin  


### Fiscal year end date dictionary

Collected from SEC EDGAR

Dictionary with Key(ticker): Value(Fiscal year end date)

Actually, the date may different every year, but we will use this for now.

In [7]:
company_fiscal_year_end = {'GIS': '0530', 'FRT': '1231', 'MMM': '1231', 'K': '1228', 'INTC': '1228', 'CAG': '0525', 'MSI': '1231', 'FIS': '1231', 'CCI': '1231', 'CDW': '1231', 'GPN': '1231', 'TMUS': '1231', 'D': '1231', 'KMI': '1231', 'NTAP': '0426', 'TSCO': '1230', 'BLK': '1231', 'CNC': '1231', 'MRK': '1231', 'JKHY': '0630', 'VTR': '1231', 'GRMN': '1228', 'AMT': '1231', 'MAR': '1231', 'HIG': '1231', 'ADI': '1102', 'ILMN': '1229', 'NKE': '0531', 'VNO': '1231', 'FITB': '1231', 'ETR': '1231', 'KIM': '1231', 'AMAT': '1027', 'OKE': '1231', 'DG': '0131', 'TFX': '1231', 'AAL': '1231', 'EXPD': '1231', 'V': '0930', 'REGN': '1231', 'PPG': '1231', 'RCL': '1231', 'VLO': '1231', 'KHC': '1228', 'HUM': '1231', 'WELL': '1231', 'ALLE': '1231', 'UNP': '1231', 'LLY': '1231', 'TXN': '1231', 'GS': '1231', 'LKQ': '1231', 'SYY': '0629', 'IPG': '1231', 'AAP': '1228', 'ECL': '1231', 'MKTX': '1231', 'C': '1231', 'ZBH': '1231', 'UAA': '0331', 'MRO': '1231', 'AON': '1231', 'SPGI': '1231', 'ORLY': '1231', 'MO': '1231', 'PM': '1231', 'BKNG': '1231', 'BKR': '1231', 'AWK': '1231', 'APTV': '1231', 'WRK': '0930', 'SWKS': '0927', 'CMCSA': '1231', 'CE': '1231', 'CB': '1231', 'WMT': '0131', 'STT': '1231', 'RSG': '1231', 'AOS': '1231', 'AAPL': '0928', 'EXR': '1231', 'UA': '0331', 'CME': '1231', 'HII': '1231', 'HOLX': '0928', 'SLB': '1231', 'EW': '1231', 'EMR': '0930', 'VRTX': '1231', 'APD': '0930', 'ADBE': '1129', 'CAT': '1231', 'PNC': '1231', 'MAA': '1231', 'AES': '1231', 'SNA': '1228', 'JPM': '1231', 'NCLH': '1231', 'PKG': '1231', 'TROW': '1231', 'HST': '1231', 'NWS': '0630', 'MA': '1231', 'DVN': '1231', 'ANSS': '1231', 'ANET': '1231', 'SNPS': '1031', 'RL': '0328', 'DISH': '1231', 'HWM': '1231', 'FTV': '1231', 'L': '1231', 'LMT': '1231', 'MAS': '1231', 'STZ': '0228', 'EMN': '1231', 'CDNS': '1231', 'EL': '0630', 'SLG': '1231', 'BDX': '0930', 'UPS': '1231', 'MS': '1231', 'IQV': '1231', 'IP': '1231', 'EOG': '1231'}

## Define Tools

### LangChain Tool Use Example

List of pre-defined tools in LangChain: https://python.langchain.com/docs/integrations/tools/

In [215]:
from langchain_core.tools import tool


@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


# Let's inspect some of the attributes associated with the tool.
print(multiply.name)
print(multiply.description)
print(multiply.args)

@tool
def add(a: int, b: int) -> int:
    """Add two integers."""
    return a + b

print(add.name)
print(add.description)
print(add.args)

multiply
Multiply two numbers.
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}
add
Add two integers.
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [216]:
tools = [add, multiply]

In [217]:
llm_with_tools = llm.bind_tools(tools)

query = "What is 3 * 12?"

llm_with_tools.invoke(query).tool_calls

[{'name': 'multiply',
  'args': {'a': 3, 'b': 12},
  'id': 'call_P7RZURkNGgCugW0IMl4uC0tU',
  'type': 'tool_call'}]

In [221]:
from langchain_core.messages import HumanMessage

query = "What is 3 * 12? Also, what is 11 + 49?"

messages = [HumanMessage(query)]

ai_msg = llm_with_tools.invoke(messages)

print(ai_msg.tool_calls)

messages.append(ai_msg)

[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_6S3NdO7nPNREYC1PpJI3GXIJ', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 11, 'b': 49}, 'id': 'call_fmZ5xe7lRpcXNtY8OP9mlydw', 'type': 'tool_call'}]


In [222]:
for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

messages

[HumanMessage(content='What is 3 * 12? Also, what is 11 + 49?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_6S3NdO7nPNREYC1PpJI3GXIJ', 'function': {'arguments': '{"a": 3, "b": 12}', 'name': 'multiply'}, 'type': 'function'}, {'id': 'call_fmZ5xe7lRpcXNtY8OP9mlydw', 'function': {'arguments': '{"a": 11, "b": 49}', 'name': 'add'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 83, 'total_tokens': 133, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_482c22a7bc', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-8266bde4-cba4-4104-af87-e193fe88fbde-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_6S3NdO7nPNREYC1PpJI3GXIJ', 'type': 'tool_call'}, {'name': 'add'

In [223]:
llm_with_tools.invoke(messages).content

'The result of \\(3 \\times 12\\) is 36, and the result of \\(11 + 49\\) is 60.'

### Extract fiscal tool

In [224]:
company_fiscal_year_end

{'GIS': '0530',
 'FRT': '1231',
 'MMM': '1231',
 'K': '1228',
 'INTC': '1228',
 'CAG': '0525',
 'MSI': '1231',
 'FIS': '1231',
 'CCI': '1231',
 'CDW': '1231',
 'GPN': '1231',
 'TMUS': '1231',
 'D': '1231',
 'KMI': '1231',
 'NTAP': '0426',
 'TSCO': '1230',
 'BLK': '1231',
 'CNC': '1231',
 'MRK': '1231',
 'JKHY': '0630',
 'VTR': '1231',
 'GRMN': '1228',
 'AMT': '1231',
 'MAR': '1231',
 'HIG': '1231',
 'ADI': '1102',
 'ILMN': '1229',
 'NKE': '0531',
 'VNO': '1231',
 'FITB': '1231',
 'ETR': '1231',
 'KIM': '1231',
 'AMAT': '1027',
 'OKE': '1231',
 'DG': '0131',
 'TFX': '1231',
 'AAL': '1231',
 'EXPD': '1231',
 'V': '0930',
 'REGN': '1231',
 'PPG': '1231',
 'RCL': '1231',
 'VLO': '1231',
 'KHC': '1228',
 'HUM': '1231',
 'WELL': '1231',
 'ALLE': '1231',
 'UNP': '1231',
 'LLY': '1231',
 'TXN': '1231',
 'GS': '1231',
 'LKQ': '1231',
 'SYY': '0629',
 'IPG': '1231',
 'AAP': '1228',
 'ECL': '1231',
 'MKTX': '1231',
 'C': '1231',
 'ZBH': '1231',
 'UAA': '0331',
 'MRO': '1231',
 'AON': '1231',
 'SP

Extract company ticker from the question

We implemented using only LLM for now, but we may need to connect DB for stability later.

In [8]:
from langchain_core.prompts import PromptTemplate

extract_ticker_prompt = PromptTemplate(
    input_variables=["question"],
    template="""
    You have to extract the company name in the given question, and answer the ticker of the company.
    Answer in JSON format, with 'ticker' key.

    Question: {question}

    Output JSON: {{
      "ticker": "<ticker of the company>"
    }}
    """
)

from pydantic import BaseModel, Field

class Ticker(BaseModel):
    """Ticker of the company in the given question"""
    ticker: str = Field(description="ticker of the company")

extract_ticker_chain = extract_ticker_prompt | llm.with_structured_output(Ticker)

In [9]:
extract_ticker_chain.invoke({"question": "What was the revenue of Apple in 2022?"})

Ticker(ticker='AAPL')

In [10]:
from langchain_core.output_parsers import StrOutputParser
from datetime import datetime

parser = StrOutputParser()

change_prompt = PromptTemplate(
    input_variables=["original_question", "fiscal_info", "today"],
    template="""
    You have to convert relative time information in the given original question to the fiscal year of the company.
    Use the fiscal end date to convert to fiscal year.
    Answer only the converted question.

    Original question: {original_question}
    Fiscal end date for the company: {fiscal_info}
    Today date: {today}
    """
)

convert_chain = change_prompt | llm | parser

def get_fiscal_info_with_ticker(ticker: str) -> str:
    """Get fiscal year information of the company with ticker string."""
    ticker = ticker.upper()
    if ticker in company_fiscal_year_end:
      end_date = company_fiscal_year_end[ticker]
      return end_date[:2] + '-' + end_date[2:]
    else:
      return '12-31'

extract_fiscal_prompt = PromptTemplate(
    input_variables=["question"],
    template="""
    You have to extract the fiscal year in the given question.
    Answer in JSON format, with 'fy' key.

    Question: {question}

    Output JSON: {{
      "fy": <fiscal year in the question>
    }}
    """
)

from pydantic import BaseModel, Field

class FiscalYear(BaseModel):
    """FiscalYear mentioned in the given question"""
    fy: int = Field(description="fiscal year in the question")

extract_fiscal_chain = extract_fiscal_prompt | llm.with_structured_output(FiscalYear)

In [16]:
from langchain_core.tools import tool

@tool
def rewrite2fiscal(question: str) -> tuple[str, int]:
  """Extract the company and convert relative time information in the question to the actual fiscal year of the company.
     It uses today function for the date information.

    Args:
        question: Question to be extracted.

    Returns:
        A tuple of ticker of the company and the fiscal year
  """
  ticker = extract_ticker_chain.invoke({"question": question}).ticker
  today = datetime.now().strftime("%Y-%m-%d")
  response = convert_chain.invoke({"original_question": question, "fiscal_info": get_fiscal_info_with_ticker(ticker), "today": today})
  fy = extract_fiscal_chain.invoke({'question': response}).fy
  return ticker, fy

In [12]:
rewrite2fiscal("What was the revenue of Apple in 2023-07-10?")

<ipython-input-12-572a0c605e94>:1: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  rewrite2fiscal("What was the revenue of Apple in 2023-07-10?")


09-28


('AAPL', 2023)

In [13]:
rewrite2fiscal("What was the revenue of Apple in 2023-12-20?")

09-28


('AAPL', 2024)

In [14]:
rewrite2fiscal("What was the revenue of Apple in last year?")

09-28


('AAPL', 2023)

In [15]:
rewrite2fiscal("What was the revenue of Apple in this year?")

09-28


('AAPL', 2024)

### Domain specific calulation tools

You should implement more tools to answer complex questions.

In [17]:
@tool
def calculate_eps(net_income: float, outstanding_shares: int):
  """Calculate the EPS of the company using net income and outstanding share

   Args:
        net_income: Net income value of the company
        outstanding_shares: Total stock held by the company's shareholders

    Returns:
        EPS value
  """
  if (net_income is None) or (outstanding_shares is None):
    print("Give me the net income and outstanding shares first")
    return None
  elif outstanding_shares == 0:
    print("Outstanding shares cannot be zero")
    return None

  return net_income / outstanding_shares

In [18]:
@tool
def calculate_cashflowfromoperations(net_income: float, non_cash_items: float, changes_in_working_capital: float):
  """Calculate the cash flow from operations of the company using net income, non cash items and change in working capital

   Args:
        net_income: Net income value of the company
        non_cash_items: Financial transactions or events that are recorded in a company's financial statements but do not involve the exchange of cash
        changes_in_working_capital: Difference in a company's working capital between two reporting periods

    Returns:
        Value of cash flow from operations
  """
  if (net_income is None) or (non_cash_items is None) or (changes_in_working_capital is None):
    print("Give me the net income, non cash items and change in working capital data first")
    return None

  return net_income + non_cash_items + changes_in_working_capital

### Retriever Tool

#### Build Vector DB

You can skip this part for demo.

In [133]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

embeddings = OpenAIEmbeddings(model='text-embedding-3-small',api_key=OPENAI_API_KEY)

# Assuming 'distinct_items_with_context' is a dictionary as shown in your code
for filename, context_data in distinct_items_with_context.items():
    pre_text = merge_text_list(context_data['pre_text'])
    post_text = merge_text_list(context_data['post_text'])
    table = context_data['table']

    # Split the pre-text and post-text into smaller chunks
    pre_text_chunks = splitter.split_text(pre_text)
    post_text_chunks = splitter.split_text(post_text)

    # Create a list of texts to store in the vector DB
    texts_for_db = []
    metadatas_for_db = []

    # Add pre-text chunks
    for chunk in pre_text_chunks:
        texts_for_db.append(chunk)
        metadatas_for_db.append({'company': filename.split('/')[0], 'fiscal': int(filename.split('/')[1]), 'context_type': 'pre_text'})

    # Add table
    if table:
        texts_for_db.append(table)
        metadatas_for_db.append({'company': filename.split('/')[0], 'fiscal': int(filename.split('/')[1]), 'context_type': 'table'})

    # Add post-text chunks
    for chunk in post_text_chunks:
        texts_for_db.append(chunk)
        metadatas_for_db.append({'company': filename.split('/')[0], 'fiscal': int(filename.split('/')[1]), 'context_type': 'post_text'})


    # Create the vector database for each file
    docsearch = Chroma.from_texts(texts_for_db, embeddings, metadatas=metadatas_for_db, persist_directory="./db")

In [134]:
docsearch.persist()

<ipython-input-134-2ebc4a538f4a>:1: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  docsearch.persist()


In [135]:
!zip -r ./db.zip ./db

  adding: db/ (stored 0%)
  adding: db/chroma.sqlite3 (deflated 62%)
  adding: db/c52496c4-34d2-4e9a-a78e-3c563b0247ff/ (stored 0%)
  adding: db/c52496c4-34d2-4e9a-a78e-3c563b0247ff/data_level0.bin (deflated 17%)
  adding: db/c52496c4-34d2-4e9a-a78e-3c563b0247ff/header.bin (deflated 54%)
  adding: db/c52496c4-34d2-4e9a-a78e-3c563b0247ff/index_metadata.pickle (deflated 43%)
  adding: db/c52496c4-34d2-4e9a-a78e-3c563b0247ff/length.bin (deflated 18%)
  adding: db/c52496c4-34d2-4e9a-a78e-3c563b0247ff/link_lists.bin (deflated 79%)


In [258]:
query = "What was the revenue of Apple in 2012?"
docs = docsearch.similarity_search(query)

for doc in docs:
  print(doc.page_content)
  print(doc.metadata)
  print('--------------')

( in millions ),2013,2012,2011
net revenue,$ 33039,$ 34504,$ 35624
operating income,$ 11827,$ 13106,$ 14840

{'company': 'INTC', 'context_type': 'table', 'fiscal': 2013}
--------------
in millions,2012,2011,2010
sales,$ 6230,$ 6215,$ 5940
operating profit,599,872,481

{'company': 'IP', 'context_type': 'table', 'fiscal': 2012}
--------------
in millions,2012,2011,2010
sales,$ 13280,$ 10430,$ 9840
operating profit,1066,1147,826

{'company': 'IP', 'context_type': 'table', 'fiscal': 2012}
--------------
in millions,2012,2011,2010
sales,$ 3170,$ 3710,$ 3400
operating profit,268,163,207

{'company': 'IP', 'context_type': 'table', 'fiscal': 2012}
--------------


#### Load Vector DB

In [19]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

embeddings = OpenAIEmbeddings(model='text-embedding-3-small',api_key=OPENAI_API_KEY)

from langchain_chroma import Chroma

docsearch = Chroma(
    persist_directory="./db",
    embedding_function=embeddings
)

<ipython-input-19-dfa477852d10>:5: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(model='text-embedding-3-small',api_key=OPENAI_API_KEY)


#### Retriever tool with filtered search

In [20]:
def retrieve(question:str, ticker: str, fy: int) -> list[str]:
  """Search vector DB for the financial reports with the question and ticker and fiscal year

   Args:
        question: Question need to be answered
        ticker: Ticker of the company for filtering the documents
        fy: Fiscal year for filtering the documents

    Returns:
        A related document for the question.
  """
  retriever = docsearch.as_retriever(search_kwargs={'k': 1, 'filter':
  {
      "$and": [
          {
              "company": {
                  "$eq": ticker
              }
          },
          {
              "fiscal": {
                  "$eq": fy
              }
          }
      ]
  }})
  result = retriever.invoke(question)
  if result:
    return result
  else:
    return ["No data returned. Try again with correct ticker and fiscal year, or different question"]

In [21]:
retrieve("What was the revenue of Apple in 2012?","AAPL", 2012)

[Document(metadata={'company': 'AAPL', 'context_type': 'post_text', 'fiscal': 2012}, page_content='the gross margin percentage in 2012 was 43.9% ( 43.9 % ) , compared to 40.5% ( 40.5 % ) in 2011 .this year-over-year increase in gross margin was largely driven by lower commodity and other product costs , a higher mix of iphone sales , and improved leverage on fixed costs from higher net sales .the increase in gross margin was partially offset by the impact of a stronger u.s .dollar .the gross margin percentage during the first half of 2012 was 45.9% ( 45.9 % ) compared to 41.4% ( 41.4 % ) during the second half of 2012 .the primary drivers of higher gross margin in the first half of 2012 compared to the second half are a higher mix of iphone sales and improved leverage on fixed costs from higher net sales .additionally , gross margin in the second half of 2012 was also affected by the introduction of new products with flat pricing that have higher cost structures and deliver greater val

In [22]:
@tool
def retrieve_factual_data(question:str, ticker: str, fy: int) -> str:
  """Search vector DB for the financial reports with the question and ticker and fiscal year

   Args:
        question: Question need to be answered
        ticker: Ticker of the company for filtering the documents
        fy: Fiscal year for filtering the documents

    Returns:
        A related document for the question.
  """
  retriever = docsearch.as_retriever(search_kwargs={'k': 1, 'filter':
  {
      "$and": [
          {
              "company": {
                  "$eq": ticker
              }
          },
          {
              "fiscal": {
                  "$eq": fy
              }
          }
      ]
  }})
  result = retriever.invoke(question)
  if result:
    return result[0].page_content
  else:
    return "No data returned. Try again with correct ticker and fiscal year, or different question"

## Agent with tools

In [23]:
tools = [rewrite2fiscal, calculate_eps, calculate_cashflowfromoperations, retrieve_factual_data]

In [24]:
from langchain_core.prompts import ChatPromptTemplate

llm_with_tools = llm.bind_tools(tools)

In [25]:
from langchain_core.messages import HumanMessage, SystemMessage
system_query = 'You are the professional agent that can use tools to answer the financial question. Output answer without selecting tool if you can answer the question with chat history.'
user_query = """For the given question, you should decide which tool you should use and arguments for the tool to answer the question.
        Question: {question}"""

In [41]:
query = "What was the trend of revenue for Intel 10 years ago?"

In [42]:
messages = [SystemMessage(content=system_query), HumanMessage(content=user_query.format(question=query))]

ai_msg = llm_with_tools.invoke(messages)
print(ai_msg.content)
while ai_msg.content == '':
  messages.append(ai_msg)
  for tool_call in ai_msg.tool_calls:
    selected_tool = {"rewrite2fiscal": rewrite2fiscal,
                     "calculate_eps": calculate_eps,
                     "calculate_cashflowfromoperations": calculate_cashflowfromoperations,
                     "retrieve_factual_data": retrieve_factual_data}[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)
  ai_msg = llm_with_tools.invoke(messages)
  print(ai_msg.content)

for message in messages:
  print(message)



The trend of revenue for Intel in 2013 showed a decrease of 2% compared to 2011. This decline was primarily attributed to $494 million in excess capacity charges and lower revenue from the PC Client Group (PCCG) and Data Center Group (DCG) platforms. Additionally, higher unit costs and reduced revenue from netbooks and multi-comm contributed to the decrease. However, this decline was partially offset by lower factory start-up costs and additional revenue from the acquisition of McAfee, which added approximately $334 million in gross margin dollars in 2012 compared to 2011.
content='You are the professional agent that can use tools to answer the financial question. Output answer without selecting tool if you can answer the question with chat history.' additional_kwargs={} response_metadata={}
content='For the given question, you should decide which tool you should use and arguments for the tool to answer the question.\n        Question: What was the trend of revenue for Intel 10 years

In [44]:
query = "How was the trend in Entergy's cash flow from operations between 2014 and 2017?"

messages = [SystemMessage(content=system_query), HumanMessage(content=user_query.format(question=query))]

ai_msg = llm_with_tools.invoke(messages)
print(ai_msg.content)
while ai_msg.content == '':
  messages.append(ai_msg)
  for tool_call in ai_msg.tool_calls:
    selected_tool = {"rewrite2fiscal": rewrite2fiscal,
                     "calculate_eps": calculate_eps,
                     "calculate_cashflowfromoperations": calculate_cashflowfromoperations,
                     "retrieve_factual_data": retrieve_factual_data}[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)
  ai_msg = llm_with_tools.invoke(messages)
  print(ai_msg.content)

for message in messages:
  print(message)



The retrieved data does not provide specific cash flow from operations figures for Entergy between 2014 and 2017. However, it does include some financial context for those years.

1. **2014**: The net revenue was $5,735 million.
2. **2015**: The data focuses on changes in net revenue but does not provide cash flow from operations.
3. **2016**: The increase in net revenue was attributed to lower operating and maintenance expenses, higher net revenue, and other income, but specific cash flow figures are not mentioned.
4. **2017**: Similar to 2016, the report discusses net revenue changes without detailing cash flow from operations.

To analyze the trend in cash flow from operations for Entergy between 2014 and 2017, you would typically look for specific cash flow statements for each year. Unfortunately, the current data does not include those specific figures. You may need to access Entergy's financial reports directly for detailed cash flow information.
content='You are the profession

In [45]:
query = "What was the EPS of Intel in 2017?"

In [47]:
query = "What was the EPS of Entergy in 2017?"

In [48]:
messages = [SystemMessage(content=system_query), HumanMessage(content=user_query.format(question=query))]

ai_msg = llm_with_tools.invoke(messages)
print(ai_msg.content)
while ai_msg.content == '':
  messages.append(ai_msg)
  for tool_call in ai_msg.tool_calls:
    selected_tool = {"rewrite2fiscal": rewrite2fiscal,
                     "calculate_eps": calculate_eps,
                     "calculate_cashflowfromoperations": calculate_cashflowfromoperations,
                     "retrieve_factual_data": retrieve_factual_data}[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)
  ai_msg = llm_with_tools.invoke(messages)
  print(ai_msg.content)

for message in messages:
  print(message)



The retrieved document does not provide the EPS for Entergy in 2017. It mentions a regulatory liability but does not include specific EPS information. Would you like me to search for additional financial reports or data to find the EPS for Entergy in 2017?
content='You are the professional agent that can use tools to answer the financial question. Output answer without selecting tool if you can answer the question with chat history.' additional_kwargs={} response_metadata={}
content='For the given question, you should decide which tool you should use and arguments for the tool to answer the question.\n        Question: What was the EPS of Entergy in 2017?' additional_kwargs={} response_metadata={}
content='' additional_kwargs={'tool_calls': [{'id': 'call_lHnBz6DqJRTuoVwE9LWBuLGp', 'function': {'arguments': '{"question":"What was the EPS of Entergy in 2017?"}', 'name': 'rewrite2fiscal'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 28,